In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
all_states = ['ACT', 'NSW', 'NT', 'QLD', 'SA', 'TAS', 'VIC', 'WA']

In [3]:
df1 = pd.read_csv('data/postcodes/australian_postcodes.csv', dtype={'locality': str, 'state': str, 'postcode': str})
df2 = pd.read_csv('data/postcodes/postcode-dataout.txt', header=None, names=['suburb', 'state', 'postcode'], dtype={'suburb': str, 'state': str, 'postcode': str})
df1['name'] = df1['locality'].str.upper()
df1['state'] = df1['state'].str.upper()
df2['name'] = df2['suburb'].str.upper()
df2['state'] = df2['state'].str.upper()
# df1 = df1[['name', 'state', 'postcode']]
# df2 = df2[['name', 'state', 'postcode']]

In [4]:
gdfk = {}
for state in all_states:
    state_lower = state.lower()
    gdfk[state] = gpd.read_file(f'data/geojson/suburb-10-{state_lower}.geojson')
    
    new_column_names = {}
    for old_column in gdfk[state].columns:
        old_column : str
        # new_column_names[column] = column.lstrip(f'{state_lower}_')
        column = old_column
        column = column.replace(f'{state_lower}_', '')
        column = column.replace(f'loca_', 'local_')
        column = column.replace(f'localit', 'locali')
        column = column.replace(f'locali', 'locality')
        new_column_names[old_column] = column

    gdfk[state] = gdfk[state].rename(columns=new_column_names)

In [5]:
for state in all_states:
    gdfk[state] = gdfk[state].to_crs('EPSG:4326')
    gdfk[state]['state'] = state

gdf = gpd.GeoDataFrame(pd.concat(gdfk.values(), ignore_index=True))


gdf.rename(columns={'locality': 'dt_locality', 'local_2' : 'name', 'local_4' : 'postcode'}, inplace=True)

# Full N/A: local_1 local_3 local_6
# No N/A: local_2 local_5 local_7
# Some N/A: local_4
sorted(gdf['local_5'].unique()), sorted(gdf['local_7'].unique()), gdf['lc_ply_pid'].is_unique, gdf['loc_pid'].is_unique, gdf['id'].is_unique

(['D', 'G', 'H'], ['1', '2', '3', '4', '5', '6', '7', '8'], False, False, True)

In [6]:
{v : gdf[gdf['local_7'] == v]['state'].unique() for v in sorted(gdf['local_7'].unique())}

{'1': array(['NSW'], dtype=object),
 '2': array(['VIC'], dtype=object),
 '3': array(['QLD'], dtype=object),
 '4': array(['SA'], dtype=object),
 '5': array(['WA'], dtype=object),
 '6': array(['TAS'], dtype=object),
 '7': array(['NT'], dtype=object),
 '8': array(['ACT'], dtype=object)}

In [7]:
{v : gdf[gdf['local_5'] == v]['state'].unique() for v in sorted(gdf['local_5'].unique())}

{'D': array(['ACT'], dtype=object),
 'G': array(['ACT', 'NSW', 'NT', 'QLD', 'SA', 'TAS', 'VIC', 'WA'], dtype=object),
 'H': array(['SA'], dtype=object)}

In [8]:
{v : len(gdf[gdf['local_5'] == v]) for v in sorted(gdf['local_5'].unique())}

{'D': 19, 'G': 15709, 'H': 560}

In [9]:
gdf = gdf[['name', 'state', 'postcode', 'local_5', 'lc_ply_pid', 'loc_pid', 'dt_create', 'dt_locality', 'id', 'geometry']]

In [10]:
gdf_temp = gdf.groupby('loc_pid')[['name', 'state', 'postcode']].nunique()
len(gdf_temp[gdf_temp['name'] > 1]), len(gdf_temp[gdf_temp['state'] > 1]), len(gdf_temp[gdf_temp['postcode'] > 1])

(0, 0, 0)

In [11]:
gdf[gdf['lc_ply_pid'].duplicated(keep=False)].sort_values('lc_ply_pid')

,name,state,postcode,local_5,lc_ply_pid,loc_pid,dt_create,dt_locality,id,geometry
15376,CASCADE,WA,None,G,10019,WA1992,2012-11-22,2013-01-23,ckan_6a0ec945_c880_4882_8a81_4dbcb85e74e5.762,"POLYGON ((120.80250 -33.54257, 120.79610 -33.5..."
12924,HATTAH,VIC,None,G,10019,VIC1137,2013-11-26,2014-01-31,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.1303,"POLYGON ((142.04406 -34.68936, 142.12141 -34.6..."
13309,FLEMINGTON,VIC,None,G,10041,VIC921,2014-08-27,2014-11-07,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.1694,"POLYGON ((144.93233 -37.78091, 144.93433 -37.7..."
3236,PELICAN,NSW,None,G,10041,NSW3205,2009-06-05,2012-02-04,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.3039,"POLYGON ((151.65114 -33.06710, 151.65104 -33.0..."
14882,NEWDEGATE,WA,None,G,10041,WA2206,2012-11-22,2013-01-23,ckan_6a0ec945_c880_4882_8a81_4dbcb85e74e5.290,"POLYGON ((119.03830 -33.00079, 119.06873 -33.0..."
...,...,...,...,...,...,...,...,...,...,...
14142,REDCASTLE,VIC,None,G,9914,VIC2150,2013-11-26,2012-10-29,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.2543,"POLYGON ((144.93215 -36.70570, 144.93215 -36.7..."
13590,LOCH SPORT,VIC,None,G,9918,VIC1519,2013-11-26,2012-04-27,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.1978,"POLYGON ((147.65210 -37.98302, 147.65698 -37.9..."
14861,NILGEN,WA,None,G,9918,WA2076,2012-11-22,2013-01-23,ckan_6a0ec945_c880_4882_8a81_4dbcb85e74e5.275,"POLYGON ((115.31369 -30.97519, 115.31110 -30.9..."
11845,NEWBURY,VIC,None,G,9938,VIC1920,2013-11-26,2012-04-27,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.254,"POLYGON ((144.26870 -37.42759, 144.27143 -37.4..."


In [12]:
df1['type'].fillna('', inplace=True)

In [13]:
# ['', 'Post Office Boxes', 'Delivery Area', 'LVR']
df1['type'].unique()

array(['', 'Post Office Boxes', 'Delivery Area', 'LVR'], dtype=object)

In [14]:
dfpc1 = df1[df1['type'].isin(['Delivery Area'])][['name', 'state', 'postcode']].groupby(['name', 'state'])['postcode'].apply(lambda x: ', '.join(x)).reset_index()
dfpc2 = df1[df1['type'].isin(['Delivery Area', ''])][['name', 'state', 'postcode']].groupby(['name', 'state'])['postcode'].apply(lambda x: ', '.join(x)).reset_index()
dfpc3 = df1[df1['type'].isin(['Delivery Area', '', 'Post Office Boxes'])][['name', 'state', 'postcode']].groupby(['name', 'state'])['postcode'].apply(lambda x: ', '.join(x)).reset_index()
dfpc4 = df1[['name', 'state', 'postcode']].groupby(['name', 'state'])['postcode'].apply(lambda x: ', '.join(x)).reset_index()
dfpc5 = df2.dropna()[['name', 'state', 'postcode']].groupby(['name', 'state'])['postcode'].apply(lambda x: ', '.join(x)).reset_index()

In [15]:
gdf2 = pd.merge(gdf, dfpc1, on=['name', 'state'], how='left', suffixes=('', '_1'))
gdf2['postcode'].fillna(gdf2['postcode_1'], inplace=True)
gdf2.drop(columns='postcode_1', inplace=True)

gdf2 = pd.merge(gdf2, dfpc2, on=['name', 'state'], how='left', suffixes=('', '_1'))
gdf2['postcode'].fillna(gdf2['postcode_1'], inplace=True)
gdf2.drop(columns='postcode_1', inplace=True)

gdf2 = pd.merge(gdf2, dfpc3, on=['name', 'state'], how='left', suffixes=('', '_1'))
gdf2['postcode'].fillna(gdf2['postcode_1'], inplace=True)
gdf2.drop(columns='postcode_1', inplace=True)

gdf2 = pd.merge(gdf2, dfpc4, on=['name', 'state'], how='left', suffixes=('', '_1'))
gdf2['postcode'].fillna(gdf2['postcode_1'], inplace=True)
gdf2.drop(columns='postcode_1', inplace=True)

gdf2 = pd.merge(gdf2, dfpc5, on=['name', 'state'], how='left', suffixes=('', '_1'))
gdf2['postcode'].fillna(gdf2['postcode_1'], inplace=True)
gdf2.drop(columns='postcode_1', inplace=True)

gdf3 = gdf2[gdf2['postcode'].isna()]

In [16]:
gdf4 = gdf2[gdf2['loc_pid'].duplicated(keep=False)].sort_values('loc_pid')

In [17]:
gdf4[gdf4['postcode'].isna()]

,name,state,postcode,local_5,lc_ply_pid,loc_pid,dt_create,dt_locality,id,geometry
3297,BLUE MOUNTAINS NATIONAL PARK,NSW,NaN,G,15511,NSW421,2011-09-15,2016-11-11,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.3094,"POLYGON ((150.30333 -33.68074, 150.30541 -33.6..."
2943,BLUE MOUNTAINS NATIONAL PARK,NSW,NaN,G,10831,NSW421,2009-11-21,2016-11-11,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.2735,"POLYGON ((150.47179 -33.72192, 150.47141 -33.7..."
4226,BLUE MOUNTAINS NATIONAL PARK,NSW,NaN,G,26442,NSW421,2016-06-07,2016-11-11,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.4226,"POLYGON ((150.54932 -33.36863, 150.54710 -33.3..."
2510,BLUE MOUNTAINS NATIONAL PARK,NSW,NaN,G,13675,NSW421,2011-02-22,2016-11-11,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.2300,"POLYGON ((150.23217 -33.78396, 150.23243 -33.7..."
2431,BLUE MOUNTAINS NATIONAL PARK,NSW,NaN,G,15510,NSW421,2011-09-15,2016-11-11,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.3093,"POLYGON ((150.45133 -33.48535, 150.45261 -33.4..."
2940,BLUE MOUNTAINS NATIONAL PARK,NSW,NaN,G,10829,NSW421,2009-11-21,2016-11-11,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.2732,"POLYGON ((150.49934 -33.71570, 150.49797 -33.7..."
3296,BLUE MOUNTAINS NATIONAL PARK,NSW,NaN,G,15507,NSW421,2011-09-15,2016-11-11,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.3091,"POLYGON ((150.50360 -33.71737, 150.50326 -33.7..."
2938,BLUE MOUNTAINS NATIONAL PARK,NSW,NaN,G,10827,NSW421,2009-11-21,2016-11-11,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.2730,"POLYGON ((150.29475 -33.65012, 150.29477 -33.6..."
3872,BLUE MOUNTAINS NATIONAL PARK,NSW,NaN,G,16352,NSW421,2011-11-22,2016-11-11,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.3704,"POLYGON ((150.47383 -33.72954, 150.47218 -33.7..."
1559,BLUE MOUNTAINS NATIONAL PARK,NSW,NaN,G,23672,NSW421,2014-12-02,2016-11-11,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.1408,"POLYGON ((150.49875 -33.71723, 150.49822 -33.7..."


In [18]:
gdf5 = gdf4.dropna(subset=['postcode'])
gdf5[gdf5['postcode'].str.contains(',')]

,name,state,postcode,local_5,lc_ply_pid,loc_pid,dt_create,dt_locality,id,geometry
3875,WOODFORD,NSW,"2463, 2778",G,16358,NSW4416,2011-11-22,2013-01-23,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.3705,"POLYGON ((150.48878 -33.72349, 150.49373 -33.7..."
214,WOODFORD,NSW,"2463, 2778",G,14178,NSW4416,2011-05-17,2013-01-23,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.77,"POLYGON ((150.48561 -33.69692, 150.48604 -33.6..."
6118,WELLESLEY ISLANDS,QLD,"4871, 4892",G,63046,QLD3103,2016-06-29,2016-11-10,ckan_6bedcb55_1b1f_457b_b092_58e88952e9f0.1030,"POLYGON ((139.29440 -16.75301, 139.29335 -16.7..."
5198,WELLESLEY ISLANDS,QLD,"4871, 4892",G,63065,QLD3103,2016-06-29,2016-11-10,ckan_6bedcb55_1b1f_457b_b092_58e88952e9f0.177,"POLYGON ((139.45895 -16.66913, 139.45817 -16.6..."
5197,WELLESLEY ISLANDS,QLD,"4871, 4892",G,63064,QLD3103,2016-06-29,2016-11-10,ckan_6bedcb55_1b1f_457b_b092_58e88952e9f0.176,"POLYGON ((139.28020 -16.26417, 139.27947 -16.2..."
5801,WELLESLEY ISLANDS,QLD,"4871, 4892",G,63033,QLD3103,2016-06-29,2016-11-10,ckan_6bedcb55_1b1f_457b_b092_58e88952e9f0.1021,"POLYGON ((139.73048 -16.50170, 139.72630 -16.4..."
5195,WELLESLEY ISLANDS,QLD,"4871, 4892",G,63061,QLD3103,2016-06-29,2016-11-10,ckan_6bedcb55_1b1f_457b_b092_58e88952e9f0.174,"POLYGON ((139.80873 -16.50270, 139.80802 -16.5..."
5199,WELLESLEY ISLANDS,QLD,"4871, 4892",G,63066,QLD3103,2016-06-29,2016-11-10,ckan_6bedcb55_1b1f_457b_b092_58e88952e9f0.178,"POLYGON ((139.47389 -16.61116, 139.47346 -16.6..."
6065,WELLESLEY ISLANDS,QLD,"4871, 4892",G,63052,QLD3103,2016-06-29,2016-11-10,ckan_6bedcb55_1b1f_457b_b092_58e88952e9f0.1096,"POLYGON ((139.25304 -16.23258, 139.25174 -16.2..."
6119,WELLESLEY ISLANDS,QLD,"4871, 4892",G,63047,QLD3103,2016-06-29,2016-11-10,ckan_6bedcb55_1b1f_457b_b092_58e88952e9f0.1031,"POLYGON ((139.27343 -16.27606, 139.27140 -16.2..."


In [19]:
gdf_postcode_count = gdf2['postcode'].str.split(',').apply(lambda x: len(x) if isinstance(x, list) else x)
gdf2[gdf_postcode_count > 2]

,name,state,postcode,local_5,lc_ply_pid,loc_pid,dt_create,dt_locality,id,geometry
4105,BOREE,NSW,"2325, 2800, 2864",G,26386,NSW517,2016-06-07,2016-08-08,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.3945,"POLYGON ((148.91462 -33.23751, 148.90420 -33.2..."
4757,KATHERINE,NT,"0850, 0851, 0852",G,603,NT147,2009-06-06,2008-10-21,ckan_12eca357_6bad_4130_9c47_eaaf4c11e039.48,"POLYGON ((132.25826 -14.46151, 132.26719 -14.4..."
4775,TENNANT CREEK,NT,"0860, 0861, 0862",G,462,NT254,2009-06-06,2008-10-21,ckan_12eca357_6bad_4130_9c47_eaaf4c11e039.66,"POLYGON ((134.21375 -19.64753, 134.22973 -19.6..."
4785,ALICE SPRINGS,NT,"0870, 0871, 0872",G,612,NT5,2009-06-06,2008-10-21,ckan_12eca357_6bad_4130_9c47_eaaf4c11e039.76,"POLYGON ((133.87774 -23.69240, 133.88508 -23.6..."
4904,WINNELLIE,NT,"0820, 0821, 0822, 0906, 0907",G,528,NT293,2009-06-06,2012-02-03,ckan_12eca357_6bad_4130_9c47_eaaf4c11e039.195,"POLYGON ((130.86001 -12.43275, 130.86126 -12.4..."
8539,PORT AUGUSTA,SA,"5700, 5701, 5710",G,17053,SA1127,2015-11-26,2016-01-25,ckan_bcfcfc9a_7c8d_479a_9bdf_b95ca66ad29a.190,"POLYGON ((137.79113 -32.47629, 137.79748 -32.4..."
9581,GERMAN CREEK,SA,"5280, 5280, 5291",G,21211,SA468,2017-06-05,2016-01-25,ckan_bcfcfc9a_7c8d_479a_9bdf_b95ca66ad29a.1213,"POLYGON ((140.57005 -37.84931, 140.56583 -37.8..."


In [ ]:
gdf2.to_file('suburbs.geojson', driver='GeoJSON')